In [25]:
import os 
from dotenv import load_dotenv
load_dotenv()
from agents import Agent, Runner,function_tool,trace
import nest_asyncio
nest_asyncio.apply()
# setting up api key
os.environ["OPENAI_API_KEY"] = os.environ['OPENAI_API_KEY_MALI']
import wikipedia
import openmeteo_requests
import requests_cache
# from retry_requests import retry
import requests
from agents.extensions.visualization import draw_graph

import json 
import requests
from serpapi import GoogleSearch
api_key = os.getenv("SERPAPI_API_KEY")


from openai.types.responses import (
    ResponseFunctionCallArgumentsDeltaEvent,  # tool call streaming
    ResponseCreatedEvent,  # start of new event like tool call or final answer
)

from tools import flight_search, yelp_search, hotels_search, wikipedia_search, get_weather
from pydantic import BaseModel
from models import first_agent_output
from agents.model_settings import ModelSettings

In [ ]:
# cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
# retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
# openmeteo = openmeteo_requests.Client(session = retry_session)


In [3]:
# agent = Agent(
#     name="Assistant",
#     instructions="""You are a talented travel agent asisstant. Who is capable of planning a trip plan, gives information about the hotels, city, food options""",
#     model='gpt-4o-mini'
# )

# # executing the ai agent
# result = Runner.run_sync(agent, """ I want to travel to Amsterdam.""")
# print(result.final_output)

In [2]:
query_stabilizer = Agent(
    name = "Query Stabilizer Asisstant Agent", 
    instructions="""You are a talented asisstant who is capable of the rewrite the user query in clear and more understandable manner. 
    Try to expand queries according to given examples.
    <examples>
    Where can I go for a beach vacation?
        beach vacation

    Book me a hotel in Paris.
        hotel
        Paris

    How much does a flight to Tokyo cost?
        flight
        Tokyo
        cost

    What are some good places to visit in Italy?
        places to visit
        Italy

    Find me a budget-friendly travel package for Europe.
        budget-friendly
        travel package
        Europe

    I’m looking for a honeymoon destination with a mix of adventure and relaxation. Preferably a tropical place with good resorts and some historical sites. Any recommendations?
        honeymoon
        adventure, relaxation
        tropical
        resorts, historical sites

    Can you help me plan a one-week trip to Japan, including flights, accommodation, and must-visit attractions? I’d like a mix of city life and traditional culture.
        one-week
        Japan
        flights, accommodation
        must-visit attractions, city life, traditional culture

    I want to book a multi-city trip across Spain and Portugal. I need recommendations on the best way to travel between cities, good places to stay, and estimated costs.
        multi-city trip
        Spain, Portugal
        travel between cities
        places to stay, cost

    I have a business trip to New York next month. I need a hotel near the financial district, and my company will cover the cost up to $300 per night. Can you find options?
        business trip
        New York
        hotel, financial district
        cost: $300 per night
    </examples>
    """,
    model='gpt-4o-mini'
    )

result = Runner.run_sync(query_stabilizer, """ I want to travel to Amsterdam.""".strip())
print(result.final_output)

travel  
Amsterdam  


In [30]:
## agents

flight_agent = Agent(
    name = "Google flight information",
    instructions= """
Returns google flight information as a json. Always answer in Turkish language. Return all the given flight information

departure_id : Parameter defines the departure airport code or location kgmid. An airport code is an uppercase 3-letter code. For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.

arrival_id : Parameter defines the arrival airport code or location kgmid. An airport code is an uppercase 3-letter code. You can search for it on Google Flights or IATA. For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.

outbound_date : Parameter defines the outbound date. The format is YYYY-MM-DD. e.g. 2025-04-09

return_date : Parameter defines the return date. The format is YYYY-MM-DD. e.g. 2025-04-15
""",
    model = 'gpt-4o-mini',
    tools=[flight_search],
    # model_settings={"temperature": 0.0}
    # output_type = first_agent_output
    model_settings=ModelSettings(temperature= 0.0,
                                 max_tokens = 4096*2)
)

yelp_agent = Agent(
    name = "YELP SEARCH",
    instructions= """Returns YELP search term results.     
    search_term : search_term
    location : location""",
    model = 'gpt-4o-mini',
    tools=[yelp_search]
)

hotel_agent = Agent(
    name = "Google hotels information",
    instructions= """Returns google hotels information as a json.     
    q : Search Query
    gl : Country""",
    model = 'gpt-4o-mini',
    tools=[hotels_search]
)

wikipedia_agent = Agent(
    name = "Wikipedia search for given destination",
    instructions= """Return the wikipedia page content for the given location.""",
    model = 'gpt-4o-mini',
    tools=[wikipedia_search]
)

weather_agent = Agent(name="Weather condition",
              instructions="You are a helpful assistant on determination of the temperature of given location. Location will be given as city name, convert that city to longitude and latitude",
              model='gpt-4o-mini',
              tools=[get_weather],
            # model_settings=ModelSettings(temperature=0.5)  # Set your desired temperature here
                                )

In [32]:
result = Runner.run_sync(flight_agent, """I want to travel from Ankara to Istanbul. The outbound date is 2025-05-05 and the return date will be 2025-05-11""".strip())
print(result.final_output)

İşte Ankara'dan İstanbul'a 2025-05-05 gidiş ve 2025-05-11 dönüş tarihli uçuş bilgileri:

### Uçuş Bilgileri

#### 1. Uçuş
- **Havayolu:** Türk Hava Yolları
- **Uçuş Numarası:** TK 2189
- **Kalkış Havaalanı:** Ankara Esenboğa Havalimanı (ESB)
- **Kalkış Zamanı:** 2025-05-05 01:50
- **Varış Havaalanı:** İstanbul Havalimanı (IST)
- **Varış Zamanı:** 2025-05-05 03:05
- **Uçuş Süresi:** 75 dakika
- **Uçak Tipi:** Airbus A321 (Sharklets)
- **Seyahat Sınıfı:** Ekonomi
- **Fiyat:** 93 USD
- **Karbon Emisyonu:** 69 kg

#### 2. Uçuş
- **Havayolu:** Türk Hava Yolları
- **Uçuş Numarası:** TK 2127
- **Kalkış Havaalanı:** Ankara Esenboğa Havalimanı (ESB)
- **Kalkış Zamanı:** 2025-05-05 10:40
- **Varış Havaalanı:** İstanbul Havalimanı (IST)
- **Varış Zamanı:** 2025-05-05 12:05
- **Uçuş Süresi:** 85 dakika
- **Uçak Tipi:** Boeing 787
- **Seyahat Sınıfı:** Ekonomi
- **Fiyat:** 93 USD
- **Karbon Emisyonu:** 48 kg

#### 3. Uçuş
- **Havayolu:** Türk Hava Yolları
- **Uçuş Numarası:** TK 2139
- **Kalkış Hav

In [ ]:
result = Runner.run_sync(yelp_agent, """I want to travel from Ankara to Amsterdam. I want to try local restaurants""".strip())
print(result.final_output)

Here are some local restaurants in Amsterdam that you might want to try:

1. **[The Pantry](https://www.yelp.com/biz/the-pantry-amsterdam?osq=local+restaurants)**  
   - **Cuisine:** Modern European  
   - **Price:** €€€  
   - **Rating:** 4.6 (590 reviews)  
   - **Neighborhood:** Centrum  
   - **Phone:** 020-6200922  
   - **Snippet:** Welcoming staff offering great service in a cozy setting.  
   ![The Pantry](https://s3-media0.fl.yelpcdn.com/bphoto/B63bIMfK4iiv_0Lm3wRjCA/348s.jpg)

2. **[Cannibale Royale](https://www.yelp.com/biz/cannibale-royale-amsterdam?osq=local+restaurants)**  
   - **Cuisine:** Brasseries  
   - **Price:** €€  
   - **Rating:** 4.3 (263 reviews)  
   - **Neighborhood:** Centrum  
   - **Phone:** 020-2337160  
   - **Snippet:** A local favorite for late-night dining.  
   ![Cannibale Royale](https://s3-media0.fl.yelpcdn.com/bphoto/0slqHXKZ1qIPLPiOZtN9IA/348s.jpg)

3. **[Door 74](https://www.yelp.com/biz/door-74-amsterdam?osq=local+restaurants)**  
   - **Cuis

In [ ]:
result = Runner.run_sync(hotel_agent, """I want to travel from Ankara to Amsterdam. I need a hotel to stay in between 2025-04-05 and the return date will be 2025-04-11""".strip())
print(result.final_output)

Here are some hotels available in Amsterdam for your stay from April 5 to April 11, 2025:

### 1. [via Amsterdam](https://www.viahostels.com/amsterdam/)
- **Description**: Funky property featuring an American eatery, a terrace & a shared kitchen, plus striking murals.
- **Check-in**: 3:00 PM | **Check-out**: 10:00 AM
- **Rate per night**: $101 
- **Total for 6 nights**: $604
- **Rating**: 4.3/5 (5,253 reviews)
- **Amenities**: Breakfast, Free Wi-Fi, Parking, Air conditioning, Bar, Restaurant, Child-friendly.
  
![via Amsterdam](https://lh4.googleusercontent.com/proxy/8MQGKugy4-YFLApZGfSIC72Q40jSAX1S8zp34TPu4qhcWbvTZQv14kdxmvfH5Y--MqLdXLVgOol-kxYHicnHdkIFc0aGKJ1MJYy3qnYQmTsJW4MeTWVTh_5Lu31TFs3iRyv6AJ9m6eRIDHIT9i-R2aLn7jmzh5k=s287-w287-h192-n-k-no-v1)

---

### 2. [MEININGER Hotel Amsterdam City West](https://www.meininger-hotels.com/en/hotels/amsterdam/hotel-amsterdam-sloterdijk/)
- **Description**: Modern hotel offering private rooms & dorms, plus a shared kitchen & a funky bar with ga

In [ ]:
result = Runner.run_sync(wikipedia_agent, """ I want to travel to Amsterdam and Leiden.""".strip())
print(result.final_output)

I encountered issues retrieving the Wikipedia content. However, I can provide some general information about Amsterdam and Leiden.

### Amsterdam
- **Capital of the Netherlands**: Known for its artistic heritage, elaborate canal system, and narrow houses with gabled facades.
- **Museums**: Home to major museums such as the Rijksmuseum (Dutch art and history), the Van Gogh Museum, and the Anne Frank House.
- **Culture**: Famous for its vibrant nightlife, flower markets, and festivals. The city's bike-friendly infrastructure makes cycling a popular way to explore.
- **Canal Ring**: A UNESCO World Heritage site consisting of 17th-century canals.

### Leiden
- **University City**: Host to the oldest university in the Netherlands, established in 1575. It's known for its significant contributions to science and culture.
- **Historic Center**: Features beautiful canals, bridges, and various historic buildings. The city has a rich history dating back to the Roman era.
- **Museums and Gardens**

In [ ]:


# Initialize your model (make sure this is defined)
model = 'gpt-4o-mini'  # Your model initialization here



result = Runner.run_sync(weather_agent, """ I want to travel to Amsterdam and Leiden.""".strip())
print(result.final_output)

The current temperatures are as follows:

- **Amsterdam:** 10.6°C
- **Leiden:** 11.4°C

If you need more information or have any other questions about your travel, feel free to ask!


In [ ]:
final_agent = Agent(
    name = 'Travel Agent',
    instructions = """You are talented travel agent. The user will give tell what she/he wants. Give information about the travel destination.
    Give information about weather, 
    give information about destinations according to custormer's interests using wikipedia
    """, 
    # model='gpt-4o-mini',
    model = 'gpt-4o',
    tools=[
            # query_stabilizer,
            get_weather, 
            wikipedia_search
            ]
    # handoffs = [weather_agent,wikipedia_agent],
    # tools = [query_stabilizer]
)

result = Runner.run_sync(final_agent, """I want to travel to Amsterdam and Leiden.""".strip())
print(result.final_output)

[non-fatal] Tracing: server error 500, retrying.


Here's some information about Amsterdam and Leiden:

### Amsterdam

**Weather:**
- Current Temperature: 14.1°C

**Overview:**
- Amsterdam is the capital city of the Netherlands, known for its artistic heritage, elaborate canal system, and narrow houses with gabled facades, which are a legacy of the city’s 17th-century Golden Age. The city is also well-known for its museums, such as the Van Gogh Museum and the Anne Frank House.

**Attractions:**
- Rijksmuseum: A Dutch national museum dedicated to arts and history.
- Van Gogh Museum: Houses the largest collection of Van Gogh's paintings and drawings.
- Anne Frank House: The biographical museum dedicated to Jewish wartime diarist Anne Frank.
- Canals of Amsterdam: The canals are emblematic of the city and a UNESCO World Heritage site.

### Leiden

**Overview:**
- Leiden is a city in the province of South Holland, known for its oldest university in the Netherlands, Leiden University, which was founded in 1575. The city boasts of historical

In [ ]:
final_agent = Agent(
    name = 'Travel Agent',
    instructions = """You are talented travel agent. The user will give tell what she/he wants. Give information about the travel destination.
    Give information about weather, 
    give information about destinations according to custormer's interests using wikipedia
    """, 
    # model='gpt-4o-mini',
    model = 'gpt-4o',
    tools=[
            # query_stabilizer,
            get_weather, 
            wikipedia_search
            ]
    # handoffs = [weather_agent,wikipedia_agent],
    # tools = [query_stabilizer]
)

result = Runner.run_sync(final_agent, """I want to travel to Amsterdam and Leiden. I am young male who likes nightlife and party""".strip())
print(result.final_output)

### Weather

- **Amsterdam**: Currently, the temperature is around 14.1°C.
- **Leiden**: Currently, the temperature is around 14.6°C.

### Amsterdam: Nightlife and Party Scene

Amsterdam is renowned for its vibrant nightlife, catering to a variety of tastes and interests:

- **Clubs and Bars**: The city is home to famous clubs like Paradiso, Melkweg, and Escape, where you can dance to music by top DJs. The Leidseplein and Rembrandtplein areas are particularly lively.

- **Live Music and Festivals**: Amsterdam hosts numerous music festivals and live events year-round. Check out smaller venues for hip-hop, jazz, and indie music.

- **Unique Experiences**: Try a night cruise along the canals for a memorable experience.

### Leiden: Nightlife and Culture

Leiden offers a more laid-back but exciting night scene:

- **Bars and Pubs**: The city is filled with cozy bars and pubs, ideal for relaxing with friends. Borrels and craft beer pubs are particularly popular.

- **Student Vibe**: As a un

In [18]:
with trace("final_agent"):
    final_agent = Agent(
    name = 'Travel Agent',
    instructions = """You are talented travel agent. The user will give tell what she/he wants. Give information about the travel destination.
    Give information about weather, 
    give information about destinations according to custormer's interests using wikipedia
    """, 
    # model='gpt-4o-mini',
    model = 'gpt-4o',
    tools=[
            # query_stabilizer,
            get_weather, 
            wikipedia_search
            ]
    # handoffs = [weather_agent,wikipedia_agent],
    # tools = [query_stabilizer]
    )

    result = Runner.run_sync(final_agent, """I want to travel to Amsterdam and Leiden. I am young male who likes nightlife and party""".strip())
    print(result.final_output)

Trace already exists. Creating a new trace, but this is probably a mistake.


### Amsterdam

**Weather:**
The current temperature in Amsterdam is approximately 11.9°C. It might be a bit chilly, so consider packing some warm clothes.

**Nightlife and Party Scene:**
Amsterdam is famous for its vibrant nightlife, offering a variety of clubs, bars, and cafes. The city is known for its open-minded atmosphere and diverse music scene. Some popular areas for nightlife include:

- **Leidseplein and Rembrandtplein:** These squares are bustling with bars, clubs, and restaurants.
- **The Jordaan:** Known for its cozy pubs and live music venues.
- **De Wallen (Red Light District):** Offers a unique mix of bars and entertainment.

### Leiden

**Nightlife and Party Scene:**
Leiden, while smaller and quieter than Amsterdam, also has a lively nightlife, particularly due to its large student population. Key areas include:

- **Pieterskwartier:** Known for its trendy bars and cafes.
- **Breestraat:** Offers a mix of pubs and bars with a more laid-back vibe.

Both cities have a ric

In [23]:
with trace("final_agent_2"):
    final_agent = Agent(
    name = 'Travel Agent',
    instructions = """You are talented travel agent. The user will give tell what she/he wants. Give information about the travel destination.
    Give information about weather, 
    give information about destinations according to custormer's interests using wikipedia
    """, 
    model='gpt-4o-mini',
    # model = 'gpt-4o',
    tools=[
            # query_stabilizer,
            get_weather, 
            wikipedia_search
            ]
    # handoffs = [weather_agent,wikipedia_agent],
    # tools = [query_stabilizer]
    )

    result = Runner.run_sync(final_agent, """I want to travel to Amsterdam and Leiden. I am young male who likes nightlife and party""".strip())
    print(result.final_output)

Trace already exists. Creating a new trace, but this is probably a mistake.


### Travel Information for Amsterdam and Leiden

**Weather:**
- **Amsterdam:** Currently around 11.9°C.
- **Leiden:** Currently around 12.6°C.

### Nightlife in Amsterdam
Amsterdam is renowned for its vibrant nightlife scene. Here are some highlights:

- **Clubs & Bars:** 
  - **De School:** A popular nightclub housed in a former school building with a great mix of techno music and a lively crowd.
  - **Club AIR:** Known for its excellent sound system and varied music genres.
  - **Café de Deen:** A lively bar with a laid-back atmosphere, perfect for starting your night out.

- **Red Light District:** Famous for its unique atmosphere, it's a place where you can find various bars and cafes. Just be mindful of the local laws and ensure respectful behavior.

- **Events and Festivals:** Depending on the time of year, you might catch events like Amsterdam Dance Event (ADE), which features the best in electronic music.

### Nightlife in Leiden
Leiden has a more relaxed vibe compared to Amste

In [7]:
result = Runner.run_sync(flight_agent, """I want to travel from Ankara(ESB) to Amsterdam(AMS). The outbound date is 2025-05-06 and the return date will be 2025-06-11""".strip())
print(result.final_output)

AttributeError: 'dict' object has no attribute 'resolve'

In [11]:
def assemble_conversation(result, new_input):
    if result !=None:
        new_input = result.to_input_list() + [{'content': new_input,
                                                'role': 'user'}]
    else:
        new_input = new_input
    return new_input

In [12]:
new_prompt = "According to your suggestions which flight is cheaper?"

In [13]:
new_input = assemble_conversation(result, new_prompt)

result = await Runner.run(flight_agent, new_input)
print(result.final_output)

İlk uçuş seçeneği Türk Hava Yolları ile 431 USD iken, ikinci uçuş seçeneği Pegasus Airlines ile 448 USD'dir. 

Bu durumda, **Türk Hava Yolları ile yapılan uçuş** (431 USD) daha ucuzdur.


In [14]:
new_prompt = "Is this price for the round trip?"
new_input = assemble_conversation(result, new_prompt)

result = await Runner.run(flight_agent, new_input)
print(result.final_output)

Evet, belirtilen fiyatlar her iki uçuş için de **gidiş-dönüş** uçuşu içindir. Yani, Ankara'dan Amsterdam'a ve ardından Amsterdam'dan Ankara'ya dönüş için toplam ücretleri kapsamaktadır.


In [ ]:
class first_agent_output(BaseModel):
    departure_id: str
    arrival_id: str
    outbound_data : str 
    return_date : str